In [19]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import spacy
import regex as re
from fuzzywuzzy import fuzz, process
import ast

In [5]:

#Cargar data frame que ya fue filtrado y limpiado para guardar entidades
df_filtrado = pd.read_csv("tweets_DEF_GOD.csv")
#Cargamos el modelo entrenado de Spacy
model102 = spacy.load("modelV1_0_2")
#Etiquetas de interés
etiquetas_deseadas = {"MUN", "CALLE", "LOC", "GPE", "FAC","COL"}
#Iniciamos listas para cada etiqueta por tweet
colonias = []
calles = []
municipios = []
locaciones = []
gpes = []
facilidades = []
#Iniciamos lista de valores actuales
col_value = []
calle_value = []
mun_value = []
loc_value = []
gpe_value = []
fac_value = []
#Procesar texto
for texto in df_filtrado["Texto"]:
    #Extraemos entidad del tweet  
    doc = model102(texto)
    #Iteramos por cada entidad identificada en forma de listas
    for ent in doc.ents:
        if ent.label_ == 'COL':
            col_value.append(ent.text)
        elif ent.label_ == 'CALLE':
            calle_value.append(ent.text)
        elif ent.label_ == 'MUN':
            mun_value.append(ent.text)
        elif ent.label_ == 'LOC':
            loc_value.append(ent.text)
        elif ent.label_ == 'GPE':
            gpe_value.append(ent.text)
        elif ent.label_ == 'FAC':
            fac_value.append(ent.text)
# Añadir los valores encontrados a las listas correspondientes (o None si no se encontraron)
    colonias.append(col_value)
    calles.append(calle_value)
    municipios.append(mun_value)
    locaciones.append(loc_value)
    gpes.append(gpe_value)
    facilidades.append(fac_value)
      #Limpiamos las listas para cada iteracion
    col_value = []
    calle_value = []
    mun_value = []
    loc_value = []
    gpe_value = []
    fac_value = []

# Añadir nuevas columnas al DataFrame para cada tipo de entidad
df_filtrado['municipio'] = municipios
df_filtrado['colonia'] = colonias
df_filtrado['calle'] = calles
df_filtrado['locacion'] = locaciones
df_filtrado['GPE'] = gpes
df_filtrado['facilidad'] = facilidades

# Guardar el DataFrame con las nuevas columnas en un archivo CSV
df_filtrado.to_csv('Entidades_etiqueta.csv', index=False)

In [29]:
# Función para normalizar el prefijo y eliminar puntos
# Agregar posibles prefijos
def normalizar_prefijos(entidad):
    entidad = entidad.lower()  # Ignorar mayúsculas/minúsculas

    # Reemplazar prefijos solo si están al inicio
    entidad = re.sub(r"^av( |$)", "avenida ", entidad)  # Reemplaza 'av' o 'ave' al inicio
    entidad = re.sub(r"^ave( |$)", "avenida ", entidad)
    entidad = re.sub(r"^col( |$)", "", entidad)  # Elimina el prefijo 'col' o 'colonia'
    entidad = re.sub(r"^colonia( |$)", "", entidad)
    entidad = re.sub(r"^municipio( |$)", "", entidad)
    entidad = re.sub(r"^mun( |$)", "", entidad)
    
    entidad = entidad.replace(".", "")
    entidad = entidad.replace(" ", "")  # Eliminar puntos
    return entidad.strip()  # Eliminar espacios en blanco innecesarios

# Función para encontrar la mejor coincidencia y porcentaje
def encontrar_mejor_coincidencia(entidad, df, etiqueta):
    # Normalizar los nombres en la base de datos antes de comparar
    # x = [nombre for nombre in df[etiqueta] if pd.notna(nombre)]
    # print(x)
    nombres_colonias_normalizados = [normalizar_prefijos(nombre) for nombre in df[etiqueta] if pd.notna(nombre)]
    nombresReal = [nombre for nombre in df[etiqueta] if pd.notna(nombre)]
    # Encontrar la mejor coincidencia y el puntaje
    mejor_coincidencia, score = process.extractOne(entidad, nombres_colonias_normalizados, scorer=fuzz.token_sort_ratio)
    # Encontrar el índice de la mejor coincidencia
    idx = nombres_colonias_normalizados.index(mejor_coincidencia)
    return mejor_coincidencia, score, idx, nombresReal[idx]

#Cargamos data frame con datos oficiales
datosExp = pd.read_csv('Entidades_etiqueta.csv')
datosMty = pd.read_csv("DataMTY.csv")
df_actualizado = pd.DataFrame(columns = datosExp.columns)

colIterableNames = ["municipio"]#, "colonia"]
for i in colIterableNames:
    for idx, name in datosExp[i].items():
        if pd.isna(name) or name == "[]":
            continue
        mejores_coincidencias = []
         # Convertir los strings en listas de Python si es posible
        try:
            convertLists = ast.literal_eval(name)  # Convierte string de lista a lista real    
        except (ValueError, SyntaxError):
            convertLists = [name]  # Si no es una lista válida, lo tratamos como un único elemento
            print(convertLists)

        for name in convertLists:
            name_normalizado = normalizar_prefijos(name)
            ## Especificos
            # Normalizar el nombre de la colonia antes de comparar
            if name_normalizado == "spgg" or name_normalizado == "sanpedro" or name_normalizado == "sp":
                name_normalizado = "sanpedrogarzagarcía"
            elif name_normalizado == "mty":
                name_normalizado = "monterrey"
            elif name_normalizado == "gpe":
                name_normalizado = "guadalupe"
                
            mejor_coincidencia, score, idx_name, nombreReal = encontrar_mejor_coincidencia(name_normalizado, datosMty, i)
            # Si la similitud es mayor al 75%, sobrescribir la colonia y agregar la geometría
            if score > 80:
                mejores_coincidencias.append(mejor_coincidencia)
                datosExp.loc[idx, i] = mejor_coincidencia  # Sobrescribir la colonia
                if i == "colonia":
                    datosExp.loc[idx, 'geometría'] = datosMty.iloc[idx_name]['geometry']  # Agregar la geometría
                df_actualizado = pd.concat([df_actualizado, datosExp.loc[[idx]]])  # Añadir la fila actualizada
                print(f"Fila {idx}: Reemplazando '{name_normalizado}' por '{nombreReal}' con {score}% de similitud")
            else:
                print(f"Fila {idx}: No se encontró coincidencia suficiente para '{name_normalizado}' (Similitud: {score}%)")
df_actualizado.to_csv('testDF.csv', index=False)

Fila 1: Reemplazando 'monterrey' por 'Monterrey' con 100% de similitud
Fila 2: Reemplazando 'sannicolás' por 'San Nicolás' con 100% de similitud
Fila 4: No se encontró coincidencia suficiente para 'marcoitnl' (Similitud: 44%)
Fila 9: No se encontró coincidencia suficiente para 'slp' (Similitud: 36%)
Fila 9: No se encontró coincidencia suficiente para 'gto' (Similitud: 36%)
Fila 10: No se encontró coincidencia suficiente para 'veracruz' (Similitud: 35%)
Fila 13: No se encontró coincidencia suficiente para 'guanajuato' (Similitud: 53%)
Fila 13: No se encontró coincidencia suficiente para 'hidalgonayarit' (Similitud: 44%)
Fila 13: No se encontró coincidencia suficiente para 'querétaro' (Similitud: 38%)
Fila 13: No se encontró coincidencia suficiente para 'zacatecas' (Similitud: 45%)
Fila 13: No se encontró coincidencia suficiente para 'chihuahua' (Similitud: 33%)
Fila 13: No se encontró coincidencia suficiente para 'colima' (Similitud: 40%)
Fila 13: No se encontró coincidencia suficiente 

In [ ]:
568